<a href="https://colab.research.google.com/github/bahron/machine_learning/blob/main/Front_View_Car.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Image Clasification - Front View Car**
**Menggunakan Dataset dari kaggle**

Link : https://www.kaggle.com/datasets/sadratmh/front-view-cars

**Mempersiapkan Dataset**

Downloan data dari kaggle

In [21]:
!wget --no-check-certificate \
    https://github.com/bahron/machine_learning/raw/e1f3cca1764ab1a8f7c457a54017b384bdbe3dfb/frontviewcar.zip \
  -O '/content/front-view-car.zip'

--2022-11-06 07:33:39--  https://github.com/bahron/machine_learning/raw/e1f3cca1764ab1a8f7c457a54017b384bdbe3dfb/frontviewcar.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/bahron/machine_learning/e1f3cca1764ab1a8f7c457a54017b384bdbe3dfb/frontviewcar.zip [following]
--2022-11-06 07:33:39--  https://raw.githubusercontent.com/bahron/machine_learning/e1f3cca1764ab1a8f7c457a54017b384bdbe3dfb/frontviewcar.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17595909 (17M) [application/zip]
Saving to: ‘/content/front-view-car.zip’

/content/front-view 100%[===================>]  16.78M  --.-KB/s  

**Ekstrak zip file**

In [23]:
import os, zipfile

local_zip = '/content/front-view-car.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content')
zip_ref.close


<bound method ZipFile.close of <zipfile.ZipFile filename='/content/front-view-car.zip' mode='r'>>

In [25]:
base_dir = '/content/data'
validation_dir = os.path.join(base_dir, 'val')
train_dir = os.path.join(base_dir, 'train')
bmw_dir = os.path.join(base_dir,'BMW')
ferrari_dir = os.path.join(base_dir, 'Ferrari')
lamborghini_dir = os.path.join(base_dir, 'Lamborghini')

In [26]:
train_BMW = os.path.join(train_dir, 'BMW')
train_Ferrari = os.path.join(train_dir, 'Ferrari')
train_Lamborghini = os.path.join(train_dir, 'Lamborghini')
val_BMW = os.path.join(validation_dir, 'BMW')
val_Ferrrari = os.path.join(validation_dir, 'Ferrari')
val_Lamborgini = os.path.join(validation_dir, 'Lamborghini')